# RAG(Retrieval Augmented Generation)
- [RAG](https://python.langchain.com/v0.1/docs/modules/data_connection/)은 *Retrieval Augmented Generation*의 약자로, **검색 기반 생성 기법**을 의미한다. 이 기법은 LLM이 특정 문서에 기반하여 보다 정확하고 신뢰할 수 있는 답변을 생성할 수 있도록 돕는다.     
- 사용자의 질문에 대해 자체적으로 구축한 데이터베이스(DB)나 외부 데이터베이스에서 질문과 관련된 문서를 검색하고, 이를 질문과 함께 LLM에 전달한다.
- LLM은 같이 전달된 문서를 바탕으로 질문에 대한 답변을 생성한다. 
- 이를 통해 LLM이 학습하지 않은 내용도 다룰 수 있으며, 잘못된 정보를 생성하는 환각 현상(*hallucination*)을 줄일 수 있다.

## RAG와 파인튜닝(Fine Tuning) 비교

### 파인튜닝(Fine Tuning)

- **정의**: 사전 학습(pre-trained)된 LLM에 특정 도메인의 데이터를 추가로 학습시켜 해당 도메인에 특화된 맞춤형 모델로 만드는 방식이다.
- **장점**
  - 특정 도메인에 최적화되어 높은 정확도와 성능을 낼 수 있다.
- **단점**
  - 모델 재학습에 많은 시간과 자원이 필요하다.
  - 새로운 정보가 반영되지 않으며, 이를 위해서는 다시 학습해야 한다.

### RAG

- **정의**: 모델을 다시 학습시키지 않고, 외부 지식 기반에서 정보를 검색하여 실시간으로 답변에 활용하는 방식이다.
- **장점**
  - 최신 정보를 쉽게 반영할 수 있다.
  - 모델을 수정하지 않아도 되므로 효율적이다.
- **단점**
  - 검색된 문서의 품질에 따라 답변의 정확성이 달라질 수 있다.
  - 검색 시스템 구축이 필요하다.

## 정리

| 항목       | 파인튜닝 | RAG |
| -------- | ---- | --- |
| 도메인 최적화  | 가능   | 제한적 |
| 최신 정보 반영 | 불가능  | 가능  |
| 구현 난이도   | 높음   | 보통  |
| 유연성      | 낮음   | 높음  |

- LLM은 학습 당시의 데이터만을 기반으로 작동하므로 최신 정보나 기업 내부 자료와 같은 특정한 지식 기반에 접근할 수 없다.
- 파인튜닝은 시간과 비용이 많이 들고 유지보수가 어렵다.
-	반면, RAG는 기존 LLM을 변경하지 않고도 외부 문서를 통해 그 한계를 보완할 수 있다.
- RAG는 특히 빠르게 변화하는 정보를 다루는 분야(예: 기술 지원, 뉴스, 법률 등)에서 유용하게 활용된다. 반면, 정적인 정보에 대해 높은 정확도가 필요한 경우에는 파인튜닝이 효과적이다.


## RAG 작동 단계
- 크게 "**정보 저장(인덱싱)**", "**검색**, **생성**"의 단계로 나눌 수 있다.
  
### 1. 정보 저장(인덱싱)
RAG는 사전에 정보를 가공하여 **벡터 데이터베이스**(Vector 저장소)에 저장해 두고, 나중에 검색할 수 있도록 준비한다. 이 단계는 다음과 같은 과정으로 이루어진다.

1. **Load (불러오기)**
   - 답변시 참조할 사전 정보를 가진 데이터들을 불러온다.
2. **Split/Chunking (문서 분할)**
   - 긴 텍스트를 일정한 길이의 작은 덩어리(*chunk*)로 나눈다.
   - 이렇게 해야 검색과 생성의 정확도를 높일 수 있다.
3. **Embedding (임베딩)**
   - 각 텍스트 조각을 **임베딩 벡터**로 변환한다.
   - 임베딩 벡터는 그 문서의 의미를 벡터화 한 것으로 질문과 유사한 문서를 찾을 때 인덱스로 사용된다.
4. **Store (저장)**
   - 임베딩된 벡터를 **벡터 데이터베이스**(벡터 저장소)에 저장한다.
   - 벡터 데이터베이스는 유사한 질문이나 문장을 빠르게 찾을 수 있도록 특화된 데이터 저장소이다.
   
![rag](figures/rag1.png)

### 2. 검색, 생성

사용자가 질문을 하면 다음과 같은 절차로 답변이 생성된다.
1. **Retrieve (검색)**
   - 사용자의 질문을 임베딩한 후, 이 질문 벡터와 유사한 context 벡터를 벡터 데이터베이스에서 검색하여 찾는다.
2. **Query (질의 생성)**
   - 벡터 데이터베이스에서 검색된 문서 조각과 사용자의 질문을 함께 **프롬프트**(prompt)로 구성하여 LLM에 전달한다.
3. **Generation (응답 생성)**
   - LLM은 받은 프롬프트에 대한 응답을 생성한다.
   
- **RAG 흐름**
  
![Retrieve and Generation](figures/rag2.png)


# Document Loader
- LLM에게 질의할 때 같이 제공할 Data들을 저장하기 위해 먼저 읽어들인다.(Load)
- 데이터 Resouce는 다양하다.
    - 데이터를 로드(load)하는 방식은 저장된 위치와 형식에 따라 다양하다. 
      - 로컬 컴퓨터(Local Computer)에 저장된 문서
        - 예: CSV, Excel, JSON, TXT 파일 등
      - 데이터베이스(Database)에 저장된 데이터셋
      - 인터넷에 존재하는 데이터
        - 예: 웹에 공개된 API, 웹 페이지에 있는 데이터, 클라우드 스토리지에 저장된 파일 등

![rag_load](figures/rag_load.png)

- 다양한 문서 형식(format)에 맞춰 읽어오는 다양한 **document loader** 들을 Langchain에서 지원한다.
    - 다양한 Resource들로 부터 데이터를 읽기 위해서는 다양한 라이브러리를 이용해 서로 다른 방법으로 읽어야 한다.
    - Langchain은 데이터를 읽는 다양한 방식의 코드를 하나의 interface로 사용 할 수 있도록 지원한다.
        - https://python.langchain.com/docs/how_to/#document-loaders
    - 다양한 3rd party library(ppt, github 등등 다양한 3rd party lib도 있음. )들과 연동해 다양한 Resource로 부터 데이터를 Loading 할 수 있다.
        - https://python.langchain.com/docs/integrations/document_loaders/
- **모든 document loader는 기본적으로 동일한 interface(사용법)로 호출할 수있다.**
- **반환타입**
    - **list[Document]**
    - Load 한 문서는 Document객체에 정보들을 넣는다. 여러 문서를 읽을 수 있기 대문에 list에 묶어서 반환한다.
        - **Document 속성**
            - page_content: 문서의 내용
            - metadata(option): 문서에 대한 메타데이터(정보)를 dict 형태로 저장한다. 
            - id(option): 문서의 고유 id
     
- **주의**
    - Langchain을 이용해 RAG를 구현할 때 **꼭 Langchain의 DocumentLoader를 사용해야 하는 것은 아니다.**
    - DocumentLoader는 데이터를 읽어오는 것을 도와주는 라이브러리일 뿐이다. 다른 라이브러리를 이용해서 읽어 들여도 상관없다. 

## 주요 Document Loader

### Text file
- TextLoader 이용

In [1]:
from langchain_community.document_loaders import TextLoader

path = "data/olympic.txt"

# 객체 생성 - 읽어들일 자원(파일)의 위치
loader = TextLoader(path, encoding='utf-8')

# Load - 읽어 오기
docs = loader.load() # 메소드 호출시 읽는다.
# loader.lazy_load() : 읽은 문서를 사용(조회)할 때 읽는다.

print(type(docs), len(docs))

<class 'list'> 1


In [2]:
print(docs[0].page_content[:100])
print(docs[0].metadata)

올림픽
올림픽(영어: Olympic Games, 프랑스어: Jeux olympiques)은 전 세계 각 대륙 각국에서 모인 수천 명의 선수가 참가해 여름과 겨울에 스포츠 경기를 하
{'source': 'data/olympic.txt'}


In [4]:
docs[0].metadata['category'] = "스포츠"
print(docs[0].metadata)

{'source': 'data/olympic.txt', 'category': '스포츠'}


### PDF
- PyPDF, Pymupdf 등 다양한 PDF 문서를 읽어들이는 파이썬의  3rd party library들을 이용해 pdf 문서를 Load 한다.
    - https://python.langchain.com/docs/integrations/document_loaders/#pdfs
- 각 PDF Loader 특징
    -  PyMuPDFLoader
        -   텍스트 뿐 아니라 이미지, 주석등의 정보를 추출하는데 성능이 좋다.
        -   PyMuPDF 라이브러리 기반
    - PyPDFLoader
        - 텍스트를 빠르게 추출 할 수있다.
        - PyPDF2 라이브러리 기반. 경량 라이브러리로 빠르고 큰 파일도 효율적으로 처리한다.
    - PDFPlumberLoader
        - 표와 같은 복잡한 구조의 데이터 처리하는데 강력한 성능을 보여준다. 텍스트, 이미지, 표 등을 모두 추출할 수 있다. 
        - PDFPlumber 라이브러리 기반
- 설치 패키지
    - DocumentLoader와 연동하는 라이브러리들을 설치 해야 한다.
    - `pip install pypdf -qU`
    - `pip install pymupdf -qU`
    - `pip install pdfplumber -qU`

In [2]:
!uv pip install pypdf pymupdf pdfplumber

Audited 3 packages in 91ms


In [32]:
from langchain_community.document_loaders import (PyPDFLoader, PyMuPDFLoader, PDFPlumberLoader)

path = "data/novel/동백꽃_김유정.pdf"
# loader = PyMuPDFLoader(path, mode="single") 
loader = PDFPlumberLoader(path) 
# mode : single - 한개 문서로 읽는다., page(default):page별로 doc를 만든다.
docs = loader.load()

Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBBox from font descriptor because None cannot be parsed as 4 floats
Could get FontBB

In [33]:
len(docs)

16

In [37]:
for doc in docs:
    doc.metadata['Author'] = '김유정'
docs[0].metadata

{'source': 'data/novel/동백꽃_김유정.pdf',
 'file_path': 'data/novel/동백꽃_김유정.pdf',
 'page': 0,
 'total_pages': 16,
 'Author': '김유정',
 'CreationDate': "D:20241124070355+00'00'",
 'Creator': 'Wikisource',
 'ModDate': "D:20241124070356+00'00'",
 'Producer': 'Wikisource',
 'Title': '동백꽃',
 'author': '김유정'}

In [38]:
print(docs[2].page_content[:500])

쌩이질을 하는 것은 다 뭐냐. 그것도 발소리를 죽여 가지고
등뒤로 살며시 와서,
"얘! 너 혼자만 일하니?"
하고 긴치 않는 수작을 하는 것이다.
어제까지도 저와 나는 이야기도 잘 않고 서로 만나도 본체
만 척하고 이렇게 점잖게 지내던 터이련만 오늘로 갑작스레
대견해졌음은 웬일인가. 항차 망아지만 한 계집애가 남 일하
는 놈 보구…….
"그럼 혼자 하지 떼루 하디?"
내가 이렇게 내배앝는 소리를 하니까,
"너 일하기 좋니?"
또는,
"한여름이나 되거든 하지 벌써 울타리를 하니?"
잔소리를 두루 늘어놓다가 남이 들을까 봐 손으로 입을 틀어
막고는 그 속에서 깔깔댄다. 별로 우스울 것도 없는데 날씨
가 풀리더니 이 놈의 계집애가 미쳤나 하고 의심하였다. 게
다가 조금 뒤에는 제 집께를 할금 할금 돌아보더니 행주치마
의 속으로 꼈던 바른손을 뽑아서 나의 턱밑으로 불쑥 내미는
것이다. 언제 구웠는 지 더운 김이 홱 끼치는 굵은 감자 세
개가 손에 뿌듯이 쥐였다.
3



### CSVLoader

In [39]:
from langchain_community.document_loaders import CSVLoader

path = "data/boston_hosing.csv"
loader = CSVLoader(path)
docs = loader.load() # 행 단위로 Document를 생성.
print(len(docs))

506


In [41]:
docs[10].metadata

{'source': 'data/boston_hosing.csv', 'row': 10}

In [42]:
print(docs[10].page_content)

CRIM: 0.22489
ZN: 12.5
INDUS: 7.87
CHAS: 0.0
NOX: 0.524
RM: 6.377
AGE: 94.3
DIS: 6.3467
RAD: 5.0
TAX: 311.0
PTRATIO: 15.2
B: 392.52
LSTAT: 20.45
MEDV: 15.0


In [43]:
!uv pip install requests beautifulsoup4 lxml

Resolved 9 packages in 106ms
Prepared 2 packages in 65ms
Installed 3 packages in 68ms
 + beautifulsoup4==4.14.3
 + lxml==6.0.2
 + soupsieve==2.8.1


### Web 문서 로드

#### WebBaseLoader를 이용해 Web 문서로딩

requests와 BeautifulSoup을 이용해 web 페이지의 내용을 크롤링해서 Document로 loading한다.

- 주요 파라미터
  - **web_paths***: str | list[str]
    - 크롤링할 대상 URL
  - **requests_kwargs**: dict
    - requests.get() 에 전달할 파라미터를 dict로 전달. (key: parameter변수명, value: 전달할 값)
    - headers, cookies, verify 등 설정 전달
  - **header_template**: dict
    - HTTP Header 에 넣을 값을 dict 로 전달.
  - **encoding**
    - requests의 응답 encoding을 설정 (bs_kwargs의 from_encoding 보다 상위에서 적용됨)
  - **bs_kwargs**
    - BeautifulSoup initializer에 전달할 파라미터를 dict로 전달. (key: parameter변수명, value: 전달할 값)
    -  주요 옵션
       - **parse_only**: 요청 페이지에서 특정 요소만 선택해서 가져오기. **SoupStrainer를 사용**한다.
         - BeautifulSoup의 `SoupStrainer` 를 이용해 페이지의 일부분만 가져오기
           - 웹 페이지를 파싱(parse, 구조 분석)할 때, 페이지 전체가 아닌 특정 부분만 필요한 경우가 많다. BeautifulSoup 라이브러리의 SoupStrainer를 사용하면, 원하는 태그나 속성이 있는 요소만 골라서 파싱할 수 있다.
           - BeautifulSoup("html문서", parse_only=Strainer객체)
               - Strainer객체에 지정된 영역에서만 내용 찾는다.
           - `SoupStrainer("태그명")`, `SoupStrainer(["태그명", "태그명"])`
             - 지정한 태그 만 조회
           - `SoupStrainer(name="태그명", attrs={속성명:속성값})`
             -  지정한 태그 중 속성명=속성값인 것만 조회
        - **from_encoding**: Encoding 설정 
          - "from_encoding":"utf-8"
   - **bs_get_text_kwargs**:
     - BeautifulSoup객체.get_text() 에 전달할 파라미터 dict로 전달. (key: parameter변수명, value: 전달할 값)
     - **RAG 구축시 `separator` 와 `strip=True` 으로 설정하는 것이 좋다.** (RAG 품질을 위해 강력히 권장되는 설정이다.)
       -  get_text() 는 기본적으로 태그를 제거하고 텍스트만 이어 붙여 반환한다. `separator=구분자문자` 를 지정하여 추출된 텍스트 요소들 사이에 원하는 구분자를 지정할 수있다. `\n` 을 구분자로 사용하면 텍스트 블록 사이에 줄바꿈이 들어가 **문단의 구조를 어느정도 살릴 수 있다.**
       -  웹 문서의 줄바꿈도 포함해서 읽기 때문에 공백과 줄바꿈이 혼재된 상태로 반환된다. `strip=True`로 설정하면 추출된 문자 앞뒤의 공백 문자들을 제거할 수있다.

In [6]:
from bs4 import BeautifulSoup as bs

html_txt = """
<html>
<body>
<p><b>제목</b><span>내용</span></p>
<p>다음문단</p>
<div>다음 내용</div>
</body>
</html>
"""
soup = bs(html_txt)
# 태크 빼고 text만 추출 -> get_text()
txt1 = soup.get_text()
print("-----기본-----")
print(txt1)

txt2 = soup.get_text(strip=True) # 좌우 공백문자(공백, 엔터) 제거
print("-----------strip=True-----")
print(txt2)

txt3 = soup.get_text(strip=True, separator="\n\n") # 각 태그의 text를 지정한 구분자로 나눈다.
print("-------strip=True, seperator='\n\n'------")
print(txt3)

-----기본-----


제목내용
다음문단
다음 내용



-----------strip=True-----
제목내용다음문단다음 내용
-------strip=True, seperator='

'------
제목

내용

다음문단

다음 내용


In [8]:
import os
# 크롬 : my user agent로 검색. 검색 결과 문자열 넣기.
os.environ['USER_AGENT'] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/143.0.0.0 Safari/537.36"


In [10]:
from langchain_community.document_loaders import WebBaseLoader

urls = [
    "https://m.sports.naver.com/basketball/article/076/0004357884",
    "https://m.sports.naver.com/basketball/article/396/0000730384"
]

loader = WebBaseLoader(
    web_path=urls,
    default_parser="lxml" # BeautifulSoup(문서, 'lxml')
)

docs = loader.load()
print(len(docs))

2


In [12]:
from pprint import pprint
pprint(docs[0].metadata)

{'language': 'ko',
 'source': 'https://m.sports.naver.com/basketball/article/076/0004357884',
 'title': '"트레이드? 나는 핫하잖아!" 쿨한 그리스 괴인 "협박성 발언은 없다! 단, 절박함은 필요" 왜 이런 멘트를?'}


In [ ]:
print(docs[0].page_content) # f12에서 태그들만 다 없앤거

"트레이드? 나는 핫하잖아!" 쿨한 그리스 괴인 "협박성 발언은 없다! 단, 절박함은 필요" 왜 이런 멘트를?NAVER스포츠뉴스엔터메뉴홈야구해외야구축구해외축구농구배구N골프일반e스포츠아웃도어NEW뉴스영상일정순위포토홈 바로가기NAVER스포츠뉴스엔터스포츠야구해외야구축구해외축구농구배구N골프일반e스포츠아웃도어콘텐츠오늘의 경기승부예측연재이슈톡대학스포츠랭킹기타고객센터공식 블로그메뉴 닫기본문 바로가기"트레이드? 나는 핫하잖아!" 쿨한 그리스 괴인 "협박성 발언은 없다! 단, 절박함은 필요" 왜 이런 멘트를?입력2025.12.21. 오후 2:06기사원문공감좋아요0슬퍼요0화나요0팬이에요0후속기사 원해요0텍스트 음성 변환 서비스본문 듣기를 종료하였습니다.글자 크기 변경공유하기<저작권자(c) 연합뉴스, 무단 전재-재배포, AI 학습 및 활용 금지><저작권자(c) 연합뉴스, 무단 전재-재배포, AI 학습 및 활용 금지>[스포츠조선 류동혁 기자] "트레이드? 나는 핫하잖아!"야니스 안테토쿤보는 자신의 잇단 트레이드 소문에 대해 쿨하게 반응했다. 미국 CBS 스포츠는 21일(이하 한국시각) '야니스 아데토쿤보는 밀워키 벅스와의 미래에 대해 추측성 발언을 하고 싶어하지 않는다고 밝혔다. 그는 소문에 대해 언급했다'고 보도했다. 아데토쿤보의 트레이드 소문은 시즌 전부터 떠돌았다. 밀워키는 시즌 아웃이 된 대미안 릴라드를 충격적 바이아웃으로 방출시킨 뒤 마일스 터너를 데려왔다. 윈 나우를 절실하게 원하는 아데토쿤보의 마음을 잡기 위해서였다. 밀워키가 부진에 빠졌고, 아데토쿤보는 부상 이후 복귀했지만, 또 다시 부상을 입었다. 그는 현지 매체와의 인터뷰에서 '여러가지 보도에 대해 알지 못한다. 내 에이전트가 밀워키 구단과 그 문제에 대해 협상을 하고 있다면 그건 에이전트의 개인적인 결정이다. 에이전트는 원하는 대로 대화를 나눌 수 있지만, 결국 에이전트가 나를 위해 일하는 것이다. 에이전트와 밀워키, 그리고 다른 구단 사이에서 여러 대화가 오갈 것이다. 내가 통제할 수 있는 부분은 아니다'라고

In [ ]:
from bs4 import SoupStrainer
# SoupStrainer
# (name="a") # a태그들
# (name="a", attr={"href":"....."}) # 태그 + 속성 조건.
# (id="tag의id") # id로 조회.

loader2 = WebBaseLoader(
    web_path=urls,
    bs_kwargs={
        "parse_only":SoupStrainer(attrs={"class":['_article_content']})
    },
    bs_get_text_kwargs={
        "separator":"\n", "strip":True
    }
    
)
docs2 = loader2.load()
len(docs2)

2

In [16]:
print(docs2[0].metadata)

{'source': 'https://m.sports.naver.com/basketball/article/076/0004357884'}


#### RecursiveUrlLoader

- 주어진 URL에서 시작하여 그 페이지 안의 내부 링크를 재귀적으로 따라가며 여러 웹 문서를 자동 수집하여 로드한다.
  - 시작 url을 요청/페이지를 파싱 한 뒤에 `<a href>` 들을 수집하고 그 페이지들을 요청/페이지 파싱을 한다. 
- WebBaseLoader가 단일 페이지(단일 URL) 단위라면 RecursiveUrlLoader는 **웹 사이트 구조 전체를 크롤링하는 전용 수집기**에 가깝다.
```bash
시작 URL
 ├─ 내부 링크 1
 │   ├─ 내부 링크 1-1
 │   └─ 내부 링크 1-2
 ├─ 내부 링크 2
 └─ 내부 링크 3
```
위 구조일때 무든 페이지를 재귀적으로 수집한다.
- 주요 파라미터
  - **url**: 시작 url
  - **max_depth**
    - 링크를 몇 단계 **깊이** 까지 따라갈지 제한
    - 사이트 폭주를 막기 위한 안전장치
      - **0**: 시작페이지만, **1**: 시작페이지 + 1차링크, **2**(기본값): 시작페이지 + 1차링크 + 2차링크
  - **exclude_dirs**: list[str]
    - 크롤링 제외 경로
    - ex) `exclude_dirs=['/login', 'signup']`
  - **prevent_outside**: bool
    - True: base_url 바깥 링크는 가져오지 않고 무시한다.
  - **base_url**: str
    - prevent_outside=True일 때 바깥링크의 기준. 없으면 `url`(시작 url)의 host가 된다. 
  - **extractor**
    - 문서 내용 추출 사용자 정의 함수
    - default는 응답 받은 페이지를 `BeautifulSoup(응답페이지).get_text()` 로 텍스트를 추출한다.
    - ````python
        def custom_extractor(html:str) ->str:
            # 웹 페이지 문서를 입력으로 받는다.
            soup = BeautifulSoup(html, 'lxml')
            return soup.select_one('article').get_text() # 원하는 항목을 추출해서 반환한다.
        
        loader = RecursiveUrlLoader(
            url=start_url,
            extractor=custom_extractor
        )    
    ```

In [27]:
from bs4 import BeautifulSoup
from langchain_community.document_loaders import RecursiveUrlLoader

def extractor(html:str)->str:
    # 전체 페이지를 받아서 원하는 부분만 parsing한 뒤에 반환.
    soup = BeautifulSoup(html, "lxml")
    body = soup.select_one("div.body")
    return body.get_text(strip=True, separator="\n") if body else soup.get_text(strip=True, separator="\n")

url = "https://docs.python.org/3"
loader = RecursiveUrlLoader(
    url = url,
    extractor=extractor,
    max_depth=2,
    prevent_outside=True, # url 외부 링크는 가져오지 않도록 한다. (default:url의 host. https://docs.python.org)
    base_url=url
)
docs = loader.load()

In [28]:
print(len(docs))

18


In [30]:
idx = 10
pprint(docs[idx].metadata)

{'content_type': 'text/html',
 'language': None,
 'source': 'https://docs.python.org/3.9/',
 'title': '3.9.24 Documentation'}


In [32]:
print(docs[idx].page_content)

Python 3.9.24 documentation
Welcome! This is the official documentation for Python 3.9.24.
Parts of the documentation:
What's new in Python 3.9?
or
all "What's new" documents
since 2.0
Tutorial
start here
Library Reference
keep this under your pillow
Language Reference
describes syntax and language elements
Python Setup and Usage
how to use Python on different platforms
Python HOWTOs
in-depth documents on specific topics
Installing Python Modules
installing from the Python Package Index & other sources
Distributing Python Modules
publishing modules for installation by others
Extending and Embedding
tutorial for C/C++ programmers
Python/C API
reference for C/C++ programmers
FAQs
frequently asked questions (with answers!)
Indices and tables:
Global Module Index
quick access to all modules
General Index
all functions, classes, terms
Glossary
the most important terms explained
Search page
search this documentation
Complete Table of Contents
lists all sections and subsections
Meta informati

### ArxivLoader
- https://github.com/lukasschwab/arxiv.py
- [arXiv-아카이브](https://arxiv.org/) 는 미국 코렐대학에서 운영하는 **무료 논문 저장소**로, 물리학, 수학, 컴퓨터 과학, 생물학, 금융, 경제 등 **과학, 금융 분야의 논문**들을 공유한다.
- `ArxivLoader` 를 사용해 원하는 주제의 논문들을 arXiv에서 가져와 load할 수 있다.
- **arXiv API**를 사용해 논문을 가져올 수 있다.
  - https://python.langchain.com/api_reference/community/document_loaders/langchain_community.document_loaders.arxiv.ArxivLoader.html
- 설치
  - `pip install langchain-community -qU`
  - `pip install arxiv -qU`



In [33]:
!uv pip install arxiv

Resolved 8 packages in 2.08s
   Building sgmllib3k==1.0.0
      Built sgmllib3k==1.0.0
Prepared 3 packages in 463ms
Installed 3 packages in 11ms
 + arxiv==2.3.1
 + feedparser==6.0.12
 + sgmllib3k==1.0.0


In [34]:
# windows
!uv pip install pip-system-certs

Resolved 2 packages in 89ms
Prepared 2 packages in 276ms
Installed 2 packages in 112ms
 + pip==25.3
 + pip-system-certs==5.3


In [3]:
import pip_system_certs

In [4]:
# arxiv lib 사용
import arxiv
# 검색 설정
search = arxiv.Search(
    query = "Advanced RAG", # 검색어
    max_results=5, # 검색 논문 최대 개수.
    sort_by=arxiv.SortCriterion.LastUpdatedDate # 정렬 기준.
)
# 정렬기준 : LastUpdatedDate-논문이 마지막으로 수정된 날짜 기준.
#           Relavance: Query와 관련성이 높은 순서
#           SubmittedDate: 논문이 처음 제출된 날짜 기준

# 검색 처리 Client
client = arxiv.Client()
results = client.results(search) # 검색 (iterator)

In [5]:
type(results) # iterator : nex(), for in
# 첫번째 것만 조회
paper = next(results)

In [6]:
# 논문 정보 출력
print(paper.title) # 제목
print(paper.authors) # 논문 저자
print(paper.authors[0].name) # Author.name : 이름 추출
print(paper.summary) # 논문 요약(초록)
print(paper.pdf_url) # arivx의 논문 URL
print(paper.get_short_id()) # arivx의 이 논문의 ID

Keypoint Counting Classifiers: Turning Vision Transformers into Self-Explainable Models Without Training
[arxiv.Result.Author('Kristoffer Wickstrøm'), arxiv.Result.Author('Teresa Dorszewski'), arxiv.Result.Author('Siyan Chen'), arxiv.Result.Author('Michael Kampffmeyer'), arxiv.Result.Author('Elisabeth Wetzer'), arxiv.Result.Author('Robert Jenssen')]
Kristoffer Wickstrøm
Current approaches for designing self-explainable models (SEMs) require complicated training procedures and specific architectures which makes them impractical. With the advance of general purpose foundation models based on Vision Transformers (ViTs), this impracticability becomes even more problematic. Therefore, new methods are necessary to provide transparency and reliability to ViT-based foundation models. In this work, we present a new method for turning any well-trained ViT-based model into a SEM without retraining, which we call Keypoint Counting Classifiers (KCCs). Recent works have shown that ViTs can automatic

In [7]:
# 논문 저장
import os
os.makedirs('data/papers', exist_ok=True)

paper.download_pdf(dirpath="data/papers", filename=f"{paper.get_short_id()}.pdf")

'data/papers\\2512.17891v1.pdf'

In [8]:
# 전체 다운로드
for paper in results:
    paper.download_pdf(dirpath="data/papers", filename=f"{paper.get_short_id()}.pdf")

In [9]:
# Langchain - ArxivLoader
from langchain_community.document_loaders import ArxivLoader

loader = ArxivLoader(
    query="RAG",
    top_k_results=10
)
docs = loader.load()

In [10]:
print(len(docs))

10


In [11]:
docs[0].metadata

{'Published': '2025-05-31',
 'Title': 'RAG-Gym: Systematic Optimization of Language Agents for Retrieval-Augmented Generation',
 'Authors': 'Guangzhi Xiong, Qiao Jin, Xiao Wang, Yin Fang, Haolin Liu, Yifan Yang, Fangyuan Chen, Zhixing Song, Dengyu Wang, Minjia Zhang, Zhiyong Lu, Aidong Zhang',
 'Summary': 'Retrieval-augmented generation (RAG) has shown great promise for knowledge-intensive tasks and recently advanced with agentic RAG, where language agents engage in multi-round interactions with external knowledge sources for adaptive information retrieval. However, existing agentic RAG methods often depend on ad-hoc prompt engineering and lack a unified optimization framework. We introduce RAG-Gym, a comprehensive platform that systematically explores three optimization dimensions: (1) prompt engineering, (2) actor tuning, and (3) critic training. For prompt engineering, we propose Re$^2$Search, a novel agent incorporating reasoning reflection that significantly outperforms standard p

In [14]:
print(docs[0].page_content) # 논문 내용.
# 다운로드 기능은 없음.


arXiv:2502.13957v2  [cs.CL]  31 May 2025
RAG-Gym: Systematic Optimization of Language
Agents for Retrieval-Augmented Generation
Guangzhi Xiong∗1, Qiao Jin∗2, Xiao Wang3, Yin Fang2, Haolin Liu1, Yifan Yang2, Fangyuan
Chen4, Zhixing Song5, Dengyu Wang6, Minjia Zhang3, Zhiyong Lu†2, and Aidong Zhang†1
1University of Virginia, 2National Institutes of Health, 3University of Illinois at Urbana Champaign,
4Dana-Farber Cancer Institute, 5University of Alabama at Birmingham, 6Yale School of Medicine
Abstract
Retrieval-augmented generation (RAG) has shown great promise for knowledge-
intensive tasks and recently advanced with agentic RAG, where language agents
engage in multi-round interactions with external knowledge sources for adaptive
information retrieval. However, existing agentic RAG methods often depend on
ad-hoc prompt engineering and lack a unified optimization framework. We in-
troduce RAG-Gym, a comprehensive platform that systematically explores three
optimization dimensions: (1) pr

### Docling
- IBM Research에서 개발한 오픈소스 문서처리 도구로 다양한 종류의 문서를 구조화된 데이터로 변환해 생성형 AI에서 활용할 수있도록 지원한다.
- **주요기능**
  - PDF, DOCX, PPTX, XLSX, HTML, 이미지 등 여러 형식을 지원
  - PDF의 **페이지 레이아웃, 읽기 순서, 표 구조, 코드, 수식** 등을 분석하여 정확하게 읽어들인다.
  - OCR을 지원하여 스캔된 PDF나 이미지에서 텍스트를 추출할 수있다.
  - 읽어들인 내용을 markdown, html, json등 다양한 형식으로 출력해준다.
- 설치 : `pip install langchain-docling ipywidgets -qU` 
- 참조
  - docling 사이트: https://github.com/docling-project/docling
  - 랭체인-docling https://python.langchain.com/docs/integrations/document_loaders/docling/

In [ ]:
!uv pip install langchain-docling transformers ipywidgets
# 딥러닝 모델 사용. gpu가 있을 경우 torch cuda 버전을 먼저 설치.


Resolved 126 packages in 4.37s
   Building antlr4-python3-runtime==4.9.3
   Building pylatexenc==2.10
      Built pylatexenc==2.10
      Built antlr4-python3-runtime==4.9.3
Prepared 39 packages in 2.09s
Uninstalled 2 packages in 88ms
Installed 48 packages in 692ms
 + accelerate==1.12.0
 + antlr4-python3-runtime==4.9.3
 + colorlog==6.10.1
 + dill==0.4.0
 + docling==2.65.0
 + docling-core==2.57.0
 + docling-ibm-models==3.10.3
 + docling-parse==4.7.2
 + et-xmlfile==2.0.0
 + faker==39.0.0
 + jsonlines==4.0.0
 + jsonref==1.1.0
 + langchain-docling==2.0.0
 + latex2mathml==3.78.1
 + markdown-it-py==4.0.0
 + marko==2.2.1
 + mdurl==0.1.2
 + mpire==2.10.2
 + multiprocess==0.70.18
 + omegaconf==2.3.0
 + opencv-python==4.11.0.86
 + openpyxl==3.1.5
 - pillow==12.0.0
 + pillow==11.3.0
 + pluggy==1.6.0
 + polyfactory==3.2.0
 + pyclipper==1.4.0
 + pylatexenc==2.10
 - pypdfium2==5.2.0
 + pypdfium2==4.30.0
 + python-docx==1.2.0
 + python-pptx==1.0.2
 + pywin32==311
 + rapidocr==3.4.5
 + rich==14.2.0
 + 

In [23]:
!uv pip install accelerate

Audited 1 package in 35ms


In [17]:
# huggingface 로그인-모델 받기 위해.
from dotenv import load_dotenv
from huggingface_hub import login

hf_key = os.getenv("HUGGINGFACE_API_LEY")
login(hf_key)

In [27]:
from langchain_docling import DoclingLoader
from langchain_docling.loader import ExportType

path = "data/papers/2507.01939v4.pdf"
loader = DoclingLoader(
    file_path=path,
    export_type=MARKDOWN
)
docs = loader.load()

NameError: name 'MARKDOWN' is not defined

### UnstructuredLoader
- 다양한 비정형 문서들을 읽어 오는 Unstrctured 를 사용해, 다양한 형식의 문서들을 load 해 RAG, 모델 파인튜닝에 적용할 수있게 한다.
  - 지원 파일 형식: "csv", "doc", "docx", "epub", "image", "md", "msg", "odt", "org", "pdf", "ppt", "pptx", "rtf", "rst", "tsv", "xlsx"
- **다양한 형식의 파일로 부터 text를 로딩**해야 할 경우 유용하다. 
- Local에 library를 설치해서 사용하거나,  Unstructured 가 제공하는 API service를 사용할 수 있다.
  - https://docs.unstructured.io
- 텍스트 파일, PDF, 이미지, HTML, XML, ms-office(word, ppt), epub 등 다양한 비정형 데이터 파일을 처리할 수 있다.
  - 설치, 지원 문서: https://docs.unstructured.io/open-source/installation/full-installation
  - Langchain 문서: https://python.langchain.com/docs/integrations/document_loaders/unstructured_file

> - UnstructuredLoader PDF Load 시 Document 분할 기준
>     -  문서의 구조와 콘텐츠를 기반으로 텍스트를 분할해 Document에 넣는다.
>     -  분할 기준
>        - 헤더(Header): 문서의 제목이나 섹션 제목 등
>        - 본문 텍스트(NarrativeText): 일반적인 문단이나 설명문
>        - 표(Table): 데이터가 표 형식으로 구성된 부분
>        - 리스트(List): 순서가 있거나 없는 목록
>        - 이미지(Image): 사진이나 그래픽 요소

#### 설치할 프로그램
- poppler
  - pdf 파일을 text로 변환하기 위해 필요한 프로그램
  - windows: https://github.com/oschwartz10612/poppler-windows/releases/ 에서 최신 버전 다운로드 후 압축 풀어서 설치.
    - 환경변수 Path에 "설치경로\Library\bin" 을 추가. (설치 후 IDE를 다시 시작한다.)
  - macOS: `brew install poppler`
  - Linux: `sudo apt-get install poppler-utils`
- tesseract-ocr
  - OCR 라이브러리로 pdf 이미지를 text로 변환하기 위해 필요한 프로그램 
  - windows: https://github.com/UB-Mannheim/tesseract/wiki 에서 다운받아 설치. 
    - 환경변수 Path에 설치 경로("C:\Program Files\Tesseract-OCR") 추가 한다. (설치 후 IDE를 다시 시작한다.)
  - macOS: `brew install tesseract`
  - linux(unbuntu): `sudo apt install tesseract-ocr`
- 설치 할 패키지
  - **libmagic 설치**
      - windows: `pip install python-magic-bin -qU`
      - macOS: `brew install libmagic`
      - linux(ubuntu): `sudo apt-get install libmagic-dev`
  - `pip install "unstructured[pdf]" -qU`
      - 문서 형식별로 sub module을 설치한다. (pdf, docx ..)
      - 모든 sub module 설치: `pip install unstructured[all-docs]`
      - https://docs.unstructured.io/open-source/installation/full-installation
  - `pip install langchain-unstructured -qU`

In [1]:
!uv pip install python-magic-bin

Resolved 1 package in 50ms
Prepared 1 package in 78ms
Installed 1 package in 24ms
 + python-magic-bin==0.4.14


In [2]:
!uv pip install unstructured[all-docs]

Resolved 122 packages in 507ms
   Building langdetect==1.0.9
      Built langdetect==1.0.9
Prepared 45 packages in 5.98s
Installed 49 packages in 3.01s
 + aiofiles==25.1.0
 + backoff==2.2.1
 + coloredlogs==15.0.1
 + contourpy==1.3.3
 + cycler==0.12.1
 + deprecated==1.3.1
 + effdet==0.4.1
 + emoji==2.15.0
 + flatbuffers==25.12.19
 + fonttools==4.61.1
 + google-api-core==2.28.1
 + google-cloud-vision==3.11.0
 + googleapis-common-protos==1.72.0
 + grpcio==1.76.0
 + grpcio-status==1.76.0
 + html5lib==1.1
 + humanfriendly==10.0
 + joblib==1.5.3
 + kiwisolver==1.4.9
 + langdetect==1.0.9
 + markdown==3.10
 + matplotlib==3.10.8
 + ml-dtypes==0.5.4
 + msoffcrypto-tool==5.4.2
 + nltk==3.9.2
 + olefile==0.47
 + onnx==1.20.0
 + onnxruntime==1.23.2
 + pdf2image==1.17.0
 + pi-heif==1.1.1
 + pikepdf==10.0.3
 + proto-plus==1.27.0
 + pycocotools==2.0.11
 + pypandoc==1.16.2
 + pyparsing==3.2.5
 + pyreadline3==3.5.4
 + python-iso639==2025.11.16
 + python-magic==0.4.27
 + python-multipart==0.0.21
 + pytho

In [3]:
!uv pip install langchain-unstructured

Resolved 41 packages in 169ms
Prepared 2 packages in 239ms
Uninstalled 1 package in 60ms
Installed 2 packages in 96ms
 + langchain-unstructured==1.0.0
 - onnxruntime==1.23.2
 + onnxruntime==1.19.2


In [ ]:
from langchain_unstructured import UnstructuredLoader

path = ['data/olympic_wiki.md', 'data/novel/메밀꽃_필_무렵_이효석.pdf']

loader = UnstructuredLoader(path)
docs = loader.load() # 문단 단위로 문서를 split 해서 Document에 넣어 제공.

In [7]:
len(docs)

244

In [8]:
docs[0].metadata

{'source': 'data/olympic_wiki.md',
 'category_depth': 0,
 'languages': ['kor'],
 'file_directory': 'data',
 'filename': 'olympic_wiki.md',
 'filetype': 'text/markdown',
 'last_modified': '2025-12-12T17:36:23',
 'category': 'Title',
 'element_id': '869efdd92ae840d110075ad507174066'}

In [16]:
print(docs[-20].page_content)

14. ↑ 念, 무엇을 하려고 하는 생각이나 마음 15. ↑ 흐뭇하게 16. ↑ ⼟房, 방에 들어가는 문 앞에 약간 높게 다진 흙바닥 17. ↑ 집안의 살림을 팔려고 나가야 할 18. ↑ 한 장날에서 다음 장날 사이를 세는 단위 19. ↑ 恒⽤, 흔히 늘 20. ↑ 사시장천(四時⻑天), 사계절 쉬지 않고 연달아 21. ↑ 견디기가 힘들고 고단하여 22. ↑ 돈망나니, 돈이라면 사족을 못 쓰고 못된 짓을 하는 사람 23. ↑ 나이로는 24. ↑ 철듦 25. ↑ "가볍게"의 방언 26. ↑ 다 자란 암말. 빈마(牝⾺)라고도 한다. 27. ↑ 어둠의 귀신, 어두워서 사물을 제대로 분간하지 못하


### Directory 내의 문서파일들 로딩
- DirectoryLoader 이용

In [20]:
# Unstructured 기반 - 관련 lib가 설치되어 있어야 한다.
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader(
    path="./data", # 문서파일들을 찾을 root directory
    glob=["*.docx", "*.pdf", "*.txt"], # 찾을 문서 파일 명의 패턴을 glob 패턴으로 지정. (생략: 모든 파일)
    recursive=True, # False: Path 경로에서만 찾는다. True: Path 하위 경로도 모두 찾는다.
    show_progress=True, # 진행 프로그래스바가 나온다.
)

docs = loader.load()

  5%|▍         | 1/21 [00:01<00:20,  1.05s/it]WARNING: Could get FontBBox from font descriptor because None cannot be parsed as 4 floats


 10%|▉         | 2/21 [00:01<00:16,  1.15it/s]WARNING: Could get FontBBox from font descriptor because None cannot be parsed as 4 floats


 14%|█▍        | 3/21 [00:02<00:16,  1.11it/s]WARNING: Could get FontBBox from font descriptor because None cannot be parsed as 4 floats


 19%|█▉        | 4/21 [00:03<00:17,  1.01s/it]WARNING: Could get FontBBox from font descriptor because None cannot be parsed as 4 floats


 24%|██▍       | 5/21 [00:05<00:17,  1.06s/it]WARNING: Could get FontBBox from font descriptor because None cannot be parsed as 4 floats


...

 67%|██████▋   | 14/21 [00:56<00:38,  5.48s/it]

 71%|███████▏  | 15/21 [00:57<00:24,  4.14s/it]

 76%|███████▌  | 16/21 [00:57<00:15,  3.09s/it]

 81%|████████  | 17/21 [00:58<00:10,  2.53s/it]

100%|██████████| 21/21 [01:01<00:00,  2.92s/it]

In [21]:
print(len(docs))

21


In [19]:
!uv pip show tqdm

Name: tqdm
Version: 4.67.1
Location: C:\Users\Playdata\Documents\SKN21_2\SKN21\10_langchain\.venv\Lib\site-packages
Requires: colorama
Required-by: docling, docling-ibm-models, huggingface-hub, mpire, nltk, openai, rapidocr, semchunk, transformers, unstructured


In [24]:
print(docs[0].page_content)

1

금 따는 콩밭

Exported from Wikisource on 2024년 11월 24일

2

위키백과에 이 글 과 관련된 자료가 있습니다. 금 따는 콩밭

🙝🙟 땅속 저 밑은 늘 음침하 다.

위키백과

고달픈 간드렛불, 맥없이 푸르끼하다.

밤과 달라서 낮엔 되우 흐릿하였다.

겉으로 황토 장벽으로 앞뒤좌우가 콕 막힌 좁직한 구뎅이. 흡사히 무덤 속같이 귀중중하다. 싸늘한 침묵, 쿠더브레한 흙내와 징그러운 냉기만이 그 속에 자욱하다.

곡괭이는 뻔질 흙을 이르집는다. 암팡스러이 내려쪼며,

퍽 퍽 퍼억.

이렇게 메떨어진 소리뿐. 그러나 간간 우수수 하고 벽이 헐 린다.

영식이는 일손을 놓고 소맷자락을 끌어당기어 얼굴의 땀을 훑는다. 이놈의 줄이 언제나 잡힐는지 기가 찼다. 흙 한줌을 집어 코밑에 바짝 들여대고 손가락으로 샅샅이 뒤져본다. 완 연히 버력은 좀 변한 듯싶다. 그러나 불통버력이 아주 다 풀 린 것도 아니었다. 밀똥버력이라야 금이 온다는데 왜 이리 안 나오는지.

곡괭이를 다시 집어든다. 땅에 무릎을 꿇고 궁뎅이를 번쩍 든 채 식식거린다. 곡괭이는 무작정 내려찍는다. 바닥에서

3

물이 스미어 무르팍이 흔건히 젖었다. 굿엎은 천판에서 흙방 울은 내리며 목덜미로 굴러든다. 어떤 때에는 웃벽의 한쪽이 떨어지며 등을 탕 때리고 부서진다.

그러나 그는 눈도 하나 깜짝하지 않는다. 금을 캔다고 콩밭 하나를 다 잡쳤다. 약이 올라서 죽을둥 살둥 눈이 뒤집힌 이 판이다. 손바닥에 침을 탁 뱉고 곡괭이 자루를 한번 꼰아잡 더니 쉴 줄 모른다.

등뒤에서는 흙 긁는 소리가 드윽드윽 난다. 아직도 버력을 다 못 친 모양. 이 자식이 일을 하나 시졸 하나. 남은 속이 바 직바직 타는데 웬 뱃심이 이리도 좋아.

영식이는 살기 띤 시선으로 고개를 돌렸다. 암 말 없이 수재 를 노려본다. 그제야 꾸물꾸물 바지게에 흙을 담고 등에 메 고 사다리를 올라간다.

굿이 풀리는지 벽이 우찔하였다. 흙이 부서져 내린다. 전날 이라면 이곳에서 아내 한번 못하고 생죽음

# Chunking (문서 분할)

![rag_split](figures/rag_split.png)

- Load 한 문서를 지정한 기준의 덩어리(chunk)로 나누는 작업을 진행한다.

## 나누는 이유
1. **임베딩 모델의 컨텍스트 길이 제한**
    - 대부분의 언어 모델은 한 번에 처리할 수 있는 토큰 수에 제한이 있다. 전체 문서를 통째로 입력하면 이 제한을 초과할 수 있어 처리가 불가능해진다.
2. **검색 정확도 향상**
    - 큰 문서 전체보다는 특정 주제나 내용을 다루는 작은 chunk가 사용자 질문과 더 정확하게 매칭된다. 예를 들어, 100페이지 매뉴얼에서 특정 기능에 대한 질문이 있을 때, 해당 기능을 설명하는 몇 개의 문단만 검색되는 것이 더 효과적이다.
    - 사용자 질문에 대해 문서의 모든 내용이 다 관련있는 것은 아니다. Chunking을 통해 가장 관련성 높은 부분만 선별적으로 활용할 수 있어 답변의 품질이 향상된다.
    - 전체 문서에는 질문과 무관한 내용들이 많이 포함되어 있어 모델이 혼란을 겪을 수 있다. 적절한 크기의 chunk는 이런 노이즈를 줄여준다.
3. **계산 효율성**
    - 벡터 유사도 계산, 임베딩 생성 등의 작업이 작은 chunk 단위로 수행될 때 더 빠르고 효율적이다. 메모리 사용량도 줄일 수 있다.

## 주요 Splitter
- **Splitter**는 문서를 분할(chunking)을 처리해주는 도구들이다. Langchain은 분할 대상, 방법에 따라 다양한 splitter를 제공한다.
- **Splitter 의 목표**
  - 가능한 한 **의미 있는 덩어리를 유지**하면서, **최대 길이(chunk_size)**를 넘지 않도록 나누기.
- https://reference.langchain.com/python/langchain_text_splitters/

### CharacterTextSplitter
가장  기본적인 Text spliter
- 한개의 구분자를 기준으로 분리한다. (default: "\n\n")
    - 분리된 조각이 chunk size 보다 작으면 다음 조각과 합칠 수 있다.
        - 합쳤을때 chuck_size 보다 크면 안 합친다. chuck_size 이내면 합친다.
    - 나누는 기준은 구분자이기 때문에 chunk_size 보다 글자수가 많을 수 있다.
- chunk size: 분리된 문서(chunk) 글자수 이내에서 분리되도록 한다.
    -  구분자를 기준으로 분리한다. 구분자를 기준으로 분리한 문서 조각이 chunk size 보다 크더라도 그대로 유지한다. 즉 chunk_size가 우선이 아니라 **seperator** 가 우선이다.
- 주요 파라미터
    - chunk_size: 각 조각의 최대 길이를 지정.
    - seperator: 구분 문자열을 지정. (default: '\n\n')
- CharacterTextSplitter는 단순 스플리터로 overlap기능을 지원하지는 않는다. 단 seperator가 빈문자열("") 일 경우에는 overlap 기능을 지원한다. overlap이란 각 이전 청크의 뒷부분의 문자열을 앞에 붙여 문맥을 유지하는 것을 말한다.
  
### RecursiveCharacterTextSplitter
- RecursiveCharacterTextSplitter는 **긴 텍스트를 지정된 최대 길이(chunk_size) 이하로 나누는 데 효과적인 텍스트 분할기**(splitter)이다.
- 여러 **구분자(separators)를 순차적으로 적용**하여, 가능한 한 자연스러운 문단/문장/단어 단위로 분할하고, 최종적으로는 크기 제한을 만족시킨다.
- 분할 기준 문자
    1. 두 개의 줄바꿈 문자 ("\n\n")
    2. 한 개의 줄바꿈 문자 ("\n")
    3. 공백 문자 (" ")
    4. 빈 문자열 ("")
- 작동 방식
    1. 먼저 가장 높은 우선순위의 구분자("\n\n")를 기준으로 분리한다.
    2. 분할된 조각 중 **chunk_size를 초과하는 조각**에 대해 다음 우선순위 구분자("\n" → " " → "")로 재귀적으로 재분할한다.
    3. 이 과정을 통해 모든 조각(chunk)이 chunk_size를 초과하지 않도록 만든다.  
- 주요 파라미터
    - chunk_size: 각 조각의 최대 길이를 지정.
    - chunk_overlap: 연속된 청크들 간의 겹치는 문자 수를 설정. 새로운 청크 생성 시 이전 청크의 마지막 부분에서 지정된 수만큼의 문자를 가져와서 새 청크의 앞부분에 포함시켜, 청크 경계에서 문맥의 연속성을 유지한다.
      - 구분자에 의해 청크가 나눠지면 정상적인 분리이므로 overlap이 적용되지 않는다.
      - 정상적 구분자로 나눌 수 없어 chunk_size에 맞춰 잘라진 경우 문맥의 연결성을 위애 overlap을 적용한다.
    - separators(list): 구분자를 지정한다. 지정하면 기본 구분자가 지정한 것으로 변경된다.

#### 메소드
- `split_documents(Iterable[Document]) : List[Document]`
    - Document 목록을 받아 split 처리한다.
- `split_text(str) : List[str]`
    - string text를 받아서 split 처리한다. 

In [38]:
from langchain_text_splitters import CharacterTextSplitter
from langchain_core.documents import Document

text = """123456789012345678901234567890123456789012345678901234567890123456789

abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ

가나다라마바사아자차카타파하

아야어여오요우유으이

abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ
"""
splitter = CharacterTextSplitter(
    chunk_size=60,
    chunk_overlap=10, # default:200, chuck_size보다 chuck_overlap이 크면 안됨.
    # separator="" # chunk_size 기준으로 나누기. => 이 때, chunk_overlap이 적용. 빈 문자열로 나눴을 때만 chunk_overlapㅗ=
)
result = splitter.split_text(text)
print(len(result))

4


In [34]:
for txt in result:
    print(len(txt), txt, sep="-")
    print("------------------------------------")

69-123456789012345678901234567890123456789012345678901234567890123456789
------------------------------------
52-abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ
------------------------------------
26-가나다라마바사아자차카타파하

아야어여오요우유으이
------------------------------------
52-abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ
------------------------------------


In [26]:
!uv pip install langchain-text-splitters

Audited 1 package in 39ms


In [41]:
# str -> Document 객체로 변환.
doc = Document(page_content=text, metadata={"category":"split"})
doc.metadata

docs2 = splitter.split_documents([doc])
len(docs2)


4

In [42]:
for doc in docs2:
    print(len(doc.page_content), doc.page_content)

69 123456789012345678901234567890123456789012345678901234567890123456789
52 abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ
26 가나다라마바사아자차카타파하

아야어여오요우유으이
52 abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ


In [46]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text2 = """1234567890123456789012345678901234567890
12345678901234567890123456789

abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ

가나다라마바사아자차카타파하

아야어여오요우유으이

abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQ RSTUVWXYZ
abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ
"""

splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    separators=["\n\n", '\n', '.', ' ', '']
)
result2 = splitter.split_text(text2)

for txt in result2:
    print(len(txt), txt, sep="-")
    print("---------------------------------")

40-1234567890123456789012345678901234567890
---------------------------------
29-12345678901234567890123456789
---------------------------------
49-abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVW
---------------------------------
13-NOPQRSTUVWXYZ
---------------------------------
26-가나다라마바사아자차카타파하

아야어여오요우유으이
---------------------------------
43-abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQ
---------------------------------
9-RSTUVWXYZ
---------------------------------
49-abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVW
---------------------------------
13-NOPQRSTUVWXYZ
---------------------------------


In [49]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
path = "data/olympic.txt"

# Loading->Split
loader = TextLoader(path, encoding="utf-8")
splitter = RecursiveCharacterTextSplitter(
    chunk_size=200, chunk_overlap=20
)

load_docs = loader.load()
docs = splitter.split_documents(load_docs)
len(docs)


137

In [51]:
print(docs[1].page_content)

올림픽(영어: Olympic Games, 프랑스어: Jeux olympiques)은 전 세계 각 대륙 각국에서 모인 수천 명의 선수가 참가해 여름과 겨울에 스포츠 경기를 하는 국제적인 대회이다. 전 세계에서 가장 큰 지구촌 최대의 스포츠 축제인 올림픽은 세계에서 가장 인지도있는 국제 행사이다. 올림픽은 2년마다 하계 올림픽과 동계 올림픽이 번갈아 열리며,


In [ ]:
docs = loader.load_and_split(splitter) # splitter를 이용해서 load와 split을 한번에 처리.


## Token 수 기준으로 나누기

- LLM 언어 모델들은 입력 토큰 수 제한이 있어서 요청시 제한 토큰수 이상의 프롬프트는 전송할 수 없다.
- 따라서 텍스트를 chunk로 분할할 때는 글자수 보다 **토큰 수를 기준으로 크기를 지정하는 것**이 좋다.  
- 토큰기반 분할은 텍스트의 의미를 유지하면서 분할하는 방식이므로 문자 기반 분할과 같이 단어가 중간잘리는 것들을 방지할 수 있다. 
- 토큰 수 계산할 때는 사용하는 언어 모델에 사용된 것과 동일한 tokenizer를 사용하는 것이 좋다.
  - 예를 들어 OpenAI의 GPT 모델을 사용할 경우 tiktoken 라이브러리를 활용하여 토큰 수를 정확하게 계산할 수 있다.

### [tiktoken](https://github.com/openai/tiktoken) tokenizer 기반 분할
- OpenAI에서 GPT 모델을 학습할 때 사용한 `BPE` 방식의 tokenizer. **OpenAI 언어모델을 사용할 경우 이것을 사용하는 것이 좀 더 정확하게  토큰dmf 계산할 수 있다.**
- Splitter.from_tiktoken_encoder() 메소드를 이용해 생성
  - `RecursiveCharacterTextSplitter.from_tiktoken_encoder()`
  - `CharacterTextSplitter.from_tiktoken_encoder()`
- 파라미터
  - encode_name: 인코딩 방식(토큰화 규칙)을 지정. OpenAI는 GPT 모델들 마다 다른 방식을 사용했다. 그래서 사용하려는 모델에 맞는 인코딩 방식을 지정해야 한다.
    - `cl100k_base`: GPT-4 및 GPT-3.5-Turbo 모델에서 사용된 방식.
    - `r50k_base:` GPT-3 모델에서 사용된 방식 
  - chunk_size, chunk_overlap, separators 파라미터 (위와 동일)
- tiktoken 설치
  - `pip install tiktoken`

### HuggingFace Tokenizer
- HuggingFace 모델을 사용할 경우 그 모델이 사용한 tokenizer를 이용해 토큰 기반으로 분할 한다.
  - 다른 tokenizer를 이용해 분할 할 경우 토큰 수 계산이 다르게 될 수있다.
- `from_huggingface_tokenizer()` 메소드를 이용.
  - 파라미터
    - tokenizer: HuggingFace tokenizer 객체
    - chunk_size, chunk_overlap, separators 파라미터 (위와 동일)
- `transformers` 라이브러리를 설치해야 한다.
  - `pip install transformers` 

## MarkdownHeaderTextSplitter
- Markdown Header 기준으로 Splitter
- Loading한 문서가 Markdown 문서이고 Header를 기준으로 문서의 내용이 나눠질때 사용.
- https://reference.langchain.com/python/langchain_text_splitters/#langchain_text_splitters.MarkdownTextSplitter

In [ ]:
text = """
# 대주제1
- 동물

## 중주제1
- 포유류

- 조류

### 소주제1
- 개
- 고양이
- 까치
- 독수리

# 대주제2
## 중주제2
- 기차
- 배
"""